<a href="https://colab.research.google.com/github/ndfcampbell/bayesian_regression_demo/blob/master/BayesianRegressionDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Regression Demonstration

This repository contains a simple example of Bayesian regression in an interactive notebook.

Copyright (C) 2022 Neill D. F. Campbell (n.campbell@bath.ac.uk) based on the Bayesian Machine Learning course at the University of Bath with material from Mike Tipping.

This material is made available under the AGPL license with an  **additional notice**: the material is to be used for non-commercial teaching purposes only and not for use in production nor by any profit-making venture.

In [ ]:
#@title Setup
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.spatial.distance as distance
#import seaborn as sns

import matplotlib as mpl
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['lines.linewidth'] = 2.5

%pylab inline
%config InlineBackend.figure_format = 'retina'

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from collections import namedtuple

Populating the interactive namespace from numpy and matplotlib


In [ ]:
#@title Regression Code
HyperParameters = namedtuple('HyperParameters', 
                             'num_basis_functions basis_lengthscale basis_variance penalty_param sigma_noise')
Dataset = namedtuple('Dataset',
                     'x y phi')

hyper_params = HyperParameters(num_basis_functions=15,
                               basis_lengthscale=1.0,
                               basis_variance=1.0,
                               penalty_param=1.0e-2,
                               sigma_noise=0.15)

def test_function(x):
        return np.sin(2.0 * x)

def make_design_matrix(x, basis_centres, lengthscale, variance):
        gramm = distance.cdist(x, basis_centres, 'sqeuclidean')
        return variance * np.exp(-0.5 * gramm / np.square(lengthscale))

def build_dataset(num, hyper_params, x=None, add_noise=True):
    def generate_input(num):
        return 1.5 * np.random.randn(num)[:, np.newaxis]

    def get_equal_basis_centres(num_basis_funcs):
        return np.linspace(-3.5, 3.5, num_basis_funcs)[:, np.newaxis]

    if x is None:
        x = generate_input(num)
    else:
        num = x.shape[0]
        assert x.shape[1] == 1

    y = test_function(x)
    if add_noise:
        y += hyper_params.sigma_noise * np.random.randn(num)[:, np.newaxis]

    phi = make_design_matrix(x, 
                             basis_centres=get_equal_basis_centres(hyper_params.num_basis_functions),
                             lengthscale=hyper_params.basis_lengthscale,
                             variance=hyper_params.basis_variance)
    
    return Dataset(x=x, y=y, phi=phi)

def plot_everything(hyper_params, num_train, num_validate, num_test, plot_type,
                    show_validation, num_samples=10):
    # Make Datasets
    np.random.seed(0)
    data_train = build_dataset(num_train, hyper_params)
    np.random.seed(1)
    data_validate = build_dataset(num_validate, hyper_params)
    np.random.seed(2)
    data_test = build_dataset(num_test, hyper_params)
    num_dense = 250
    x_dense = np.linspace(-3.5, 3.5, num_dense)[:, np.newaxis]
    data_dense = build_dataset(num_dense, hyper_params, 
                               x=x_dense, add_noise=False)

    if show_validation:
        plt.subplot(1, 2, 1)

    plt.xlim(-3.5, 3.5)
    plt.ylim(-4.0, 2.0)
    plt.grid(True)
    
    plt.plot(data_train.x, data_train.y, 'o', label='Training Data')
    plt.plot(data_dense.x, data_dense.y, ':', label='True Function')

    w = np.ones([hyper_params.num_basis_functions])
    label = ''
    basis = data_dense.phi
    alpha = hyper_params.penalty_param / np.square(hyper_params.sigma_noise)

    if plot_type == 'least_squares':
        w = np.linalg.solve(data_train.phi.T @ data_train.phi, data_train.phi.T @ data_train.y)
        plt.plot(data_dense.x, data_dense.phi @ w, '-', label='Least Squares')
    elif plot_type == 'penalised_least_squares':
        w = np.linalg.solve(data_train.phi.T @ data_train.phi + 
                            hyper_params.penalty_param * np.eye(hyper_params.num_basis_functions), 
                            data_train.phi.T @ data_train.y)
        plt.plot(data_dense.x, data_dense.phi @ w, '-', label='Penalised Least Squares')
    elif plot_type == 'bayesian':
        w = np.linalg.solve(data_train.phi.T @ data_train.phi + 
                            hyper_params.penalty_param * np.eye(hyper_params.num_basis_functions), 
                            data_train.phi.T @ data_train.y)
        bayesian_term = np.linalg.solve(data_train.phi.T @ data_train.phi + 
                                        np.square(hyper_params.sigma_noise) * alpha * np.eye(hyper_params.num_basis_functions),
                                        np.eye(hyper_params.num_basis_functions))
        mean_out = data_dense.phi @ bayesian_term @ data_train.phi.T @ data_train.y
        covar_out = np.square(hyper_params.sigma_noise) * data_dense.phi @ bayesian_term @ data_dense.phi.T
        var_out = np.diag(covar_out)
        line = plt.plot(data_dense.x, mean_out, '-', label='Bayesian Prediction')
        color = line[0].get_color()
        plt.fill_between(data_dense.x.flatten(), 
                         mean_out.flatten() - 2.0 * (np.sqrt(var_out) + hyper_params.sigma_noise), 
                         mean_out.flatten() + 2.0 * (np.sqrt(var_out) + hyper_params.sigma_noise), 
                         alpha=0.25, color=color)
        plt.fill_between(data_dense.x.flatten(), 
                         mean_out.flatten() - 2.0 * (hyper_params.sigma_noise), 
                         mean_out.flatten() + 2.0 * (hyper_params.sigma_noise), 
                         alpha=0.25, color=color)
        chol_cov = np.linalg.cholesky(covar_out + 1.0e-6 * np.eye(covar_out.shape[0]))
        if num_samples > 0:
            plt.plot(data_dense.x, mean_out + chol_cov @ np.random.randn(data_dense.x.shape[0], 1), 
                    color='gray', alpha=0.3, label='Samples')
        if num_samples > 1:
            plt.plot(data_dense.x, mean_out + chol_cov @ np.random.randn(data_dense.x.shape[0], num_samples - 1), 
                    color='gray', alpha=0.3)
    elif plot_type == 'gp':
        k_xx = make_design_matrix(data_train.x, data_train.x, 
                                  hyper_params.basis_lengthscale,
                                  1.0 / alpha)
        k_tx = make_design_matrix(data_dense.x, data_train.x, 
                                  hyper_params.basis_lengthscale,
                                  1.0 / alpha)
        k_tt = make_design_matrix(data_dense.x, data_dense.x, 
                                  hyper_params.basis_lengthscale,
                                  1.0 / alpha)
        k_xx_plus_noise = k_xx + np.square(hyper_params.sigma_noise) * np.eye(data_train.x.shape[0])

        mean_out = k_tx @ np.linalg.solve(k_xx_plus_noise, data_train.y)
        covar_out = k_tt - k_tx @ np.linalg.solve(k_xx_plus_noise, k_tx.T)
        var_out = np.diag(covar_out)
        line = plt.plot(data_dense.x, mean_out, '-', label='GP Prediction')
        color = line[0].get_color()
        plt.fill_between(data_dense.x.flatten(), 
                        mean_out.flatten() - 2.0 * (np.sqrt(var_out) + hyper_params.sigma_noise), 
                        mean_out.flatten() + 2.0 * (np.sqrt(var_out) + hyper_params.sigma_noise), 
                        alpha=0.25, color=color)
        plt.fill_between(data_dense.x.flatten(), 
                        mean_out.flatten() - 2.0 * (hyper_params.sigma_noise), 
                        mean_out.flatten() + 2.0 * (hyper_params.sigma_noise), 
                        alpha=0.25, color=color)
        chol_cov = np.linalg.cholesky(covar_out + 1.0e-6 * np.eye(covar_out.shape[0]))
        if num_samples > 0:
            plt.plot(data_dense.x, mean_out + chol_cov @ np.random.randn(data_dense.x.shape[0], 1), 
                    color='gray', alpha=0.3, label='Samples')
        if num_samples > 1:
            plt.plot(data_dense.x, mean_out + chol_cov @ np.random.randn(data_dense.x.shape[0], num_samples - 1), 
                 color='gray', alpha=0.3)
        
        basis = alpha * k_tx
        w = np.linalg.solve(k_xx_plus_noise, data_train.y)

    plt.legend()

    values = 1.0 * w.flatten()
    values /= np.max(np.abs([values.max(), values.min()]))
    values = 0.5 + 0.5 * values
    cm = plt.cm.coolwarm(values)
    ax = plt.gca()
    ax.set_prop_cycle('color', list(cm))
    lines = plt.plot(data_dense.x, 2.0 * basis - 4.0, '-')
    ax.set_prop_cycle(None)

    if show_validation:
        plt.subplot(1, 2, 2)
        
        penalty_variances = np.logspace(-4.0, 2.0, 75)

        training_error = []
        validation_error = []
        testing_error = []
        marginal_log_like = []
        gp_mll = []

        for pv in penalty_variances:
            w_penalty_least_squares = np.linalg.solve(data_train.phi.T @ data_train.phi + 
                                                      pv * np.eye(hyper_params.num_basis_functions), 
                                                      data_train.phi.T @ data_train.y)
            
            train_error = np.mean(np.square(data_train.y - data_train.phi @ w_penalty_least_squares))
            training_error.append(train_error)
            
            valid_error = np.mean(np.square(data_validate.y - data_validate.phi @ w_penalty_least_squares))
            validation_error.append(valid_error)
            
            test_error = np.mean(np.square(data_test.y - data_test.phi @ w_penalty_least_squares))
            testing_error.append(test_error)
            
            alpha = pv / np.square(hyper_params.sigma_noise)

            cov = (1.0 / alpha) * data_train.phi @ data_train.phi.T + np.square(hyper_params.sigma_noise) * np.eye(data_train.y.shape[0])
            ev = stats.multivariate_normal.logpdf(data_train.y.flatten(),
                                                mean=np.zeros_like(data_train.y.flatten()), 
                                                cov=cov)
            marginal_log_like.append(ev)

            if plot_type == 'gp':
                k_xx = make_design_matrix(data_train.x, data_train.x, 
                                        hyper_params.basis_lengthscale,
                                        1.0 / alpha)
                gp_mll.append(stats.multivariate_normal.logpdf(
                    data_train.y.flatten(),
                    mean=np.zeros_like(data_train.y.flatten()), 
                    cov=k_xx + np.square(hyper_params.sigma_noise) * np.eye(data_train.y.shape[0])))
            
        plt.loglog(penalty_variances, training_error, '-', label='Training Error (N = %d)' % data_train.x.shape[0])
        plt.loglog(penalty_variances, validation_error, '-', label='Validation Error (N = %d)' % data_validate.x.shape[0])
        plt.loglog(penalty_variances, testing_error, '-', label='Test Error (N = %d)' % data_test.x.shape[0])
        ax = plt.gca()
        ax.set_xlabel('Penalty Parameter')
        ax.set_ylabel('Mean Squared Error')

        ax2 = ax.twinx()
        if plot_type == 'bayesian' or plot_type == 'gp':
            ax2.semilogx(penalty_variances, -np.array(marginal_log_like), '-', label='Bayesian Evidence (N = %d)' % num_train, color='orange')
            ax2.set_ylabel('Negative Marginal Log Likelihood')
        if plot_type == 'gp':
            ax2.semilogx(penalty_variances, -np.array(gp_mll), '-', label='GP Evidence (N = %d)' % num_train, color='teal')
            ax2.set_ylabel('Negative Marginal Log Likelihood')

        plt.grid(True)

        h1, l1 = ax.get_legend_handles_labels()
        h2, l2 = ax2.get_legend_handles_labels()
        plt.legend(h1 + h2, l1 + l2, loc='lower right', bbox_to_anchor=(1,0), bbox_transform=ax.transAxes)

        y0, y1 = ax.get_ylim()
        ax.plot([hyper_params.penalty_param, hyper_params.penalty_param], 
                [y0, y1], ':k')

        testing_error = np.array(testing_error)
        best_penalty_variance = penalty_variances[np.argmin(testing_error)]

    

In [ ]:
#@title Interactive Viewer
@interact(plot_type=[('Training Data', 'training_data'),
                     ('Least Squares', 'least_squares'), 
                     ('Penalised Least Squares', 'penalised_least_squares'), 
                     ('Bayesian Regression', 'bayesian'), 
                     ('Gaussian Process', 'gp')], 
          show_validation=widgets.Checkbox(value=False, description='Show Validation Plot'),
          show_samples=widgets.Checkbox(value=False, description='Show Samples'),
          penalty_param=widgets.FloatLogSlider(value=0.01, 
                                               base=10,
                                               min=-4.0, # max exponent of base
                                               max=2.0, # min exponent of base
                                               step=0.2, # exponent step
                                               description='Penalty'),
          num_basis_functions=widgets.IntSlider(value=15,
                                                min=1,
                                                max=150,
                                                step=1,
                                                description='Num Basis'),
          num_train=widgets.IntSlider(value=15,
                                      min=1,
                                      max=150,
                                      step=1,
                                      description='Num Train'),
          num_validate=widgets.IntSlider(value=50,
                                      min=1,
                                      max=150,
                                      step=1,
                                      description='Num Validate'),
          sigma_noise=widgets.FloatSlider(value=0.15,
                                                min=0.01,
                                                max=0.8,
                                                step=0.01,
                                                description='Noise Std'))
def show(plot_type, show_validation, show_samples,
         penalty_param, num_train, num_validate, num_basis_functions,
         sigma_noise):
    hyper_params = HyperParameters(num_basis_functions=num_basis_functions,
                                   basis_lengthscale=1.0,
                                   basis_variance=1.0,
                                   penalty_param=penalty_param,
                                   sigma_noise=sigma_noise)
    plt.figure(1, figsize=[16, 6])
    plot_everything(hyper_params=hyper_params, 
                    num_train=num_train, num_validate=num_validate, num_test=500, 
                    plot_type=plot_type, show_validation=show_validation,
                    num_samples=10 if show_samples else 0)
    plt.show()

interactive(children=(Dropdown(description='plot_type', options=(('Training Data', 'training_data'), ('Least S…

# Linear (in the parameters) Regression Example


- Example dataset generated as a noisy sine function $f(x) = \sin(2 x)$

$$y_n = \sin(2 x_n) + \epsilon_n, \text{where } \epsilon \sim \mathcal{N}(0, \sigma_{\mathrm{noise}}^{2}), \text{and } n = 1 \dots N$$

- We will model $f(x)$ with a parameterised function $y(x ; \mathbf{w})$ using a set of $M$ fixed basis functions $\phi_m(x)$ such that

$$y(x ; \mathbf{w}) = \sum_{m=1}^{M} w_m \, \phi_m(x)$$

- $\mathbf{w} = (w_1, w_2, \dots, w_M)$ is a vector of model parameters (or weights) that will be modified to fit the data
- Although $y(x)$ expresses a non-linear function of the input, it is a linear function of these parameters (hence a generalisation of linear regression)
- We are using radial basis functions (RBF), that is:

$$\phi_m(x) := \exp \left( \frac{-(x - c_m)^2}{2 \ell^2} \right)$$

- This model as *hyperparameters* as the number of basis functions ($M$), the basis function locations ($\{c_m\}$) and the lengthscale ($\ell$)

# Classic Least Squares

> **Aside:** This is "similar" in concept to fitting with a neural network (or Multi-Layer Perceptron) and an L2 "loss"

- We wish to minimise the squared error between our model's predictions and the observed training data

$$E_{\mathrm{LS}}(\mathbf{w}) := \frac{1}{2} \sum_{n=1}^{N} \big(y_n - \sum_{m=1}^{M} w_m \phi_m(x_n) \big)^2$$

- We can specify the vector of observations $\mathbf{y} := (y_1, y_2, \dots, y_N)$ and the *design matrix* $\boldsymbol{\Phi}$ as

$$\boldsymbol{\Phi} \in \mathbb{R}^{N \times M}, \text{where } [\boldsymbol{\Phi}]_{nm} = \phi_m(x_n)$$

- This gives the error measure as

$$E_{\mathrm{LS}}(\mathbf{w}) := \frac{1}{2} \big\| \mathbf{y} - \boldsymbol{\Phi} \mathbf{w} \big\|^{2}_{2} $$

- Finding a stationary point by setting the derivatives to zero gives the closed form optimal parameter as

$$\hat{\mathbf{w}}_{\mathrm{LS}} = \big( \boldsymbol{\Phi}^{\top} \boldsymbol{\Phi} \big)^{-1} \boldsymbol{\Phi}^{\top} \mathbf{y} $$

> **Note:** we need to be careful about the model being under constrained if we have fewer observations than parameters (i.e. if $N < M$). The matrix $\big( \boldsymbol{\Phi}^{\top} \boldsymbol{\Phi} \big)$ may be ill-conditioned.

- Considering the **demo** above, we usually observe *over-fitting* for this model.

- To deal with this, we need to introduce some form of complexity control (or *regularisation*) to encourage the solution to display our preference for *smooth functions* (a **prior**!)...

> **Aside:** Over-fitting and generalisation are interesting and difficult topics that are often discussed at high-level without many specifics. This discussion can partly be phrased in a *bias-variance* trade-off but parts of it are deeper than that. There is an interesting discussion on Sebastian Nowozin' Blog in a post entitled ["Do Bayesians Overfit?"](http://www.nowozin.net/sebastian/blog/do-bayesians-overfit.html) which provides an introduction that probes more deeply. We also have ignored the topic of *model specification* that also plays an important part.

# Penalised Least Squares

- We can express a preference for smoother functions in our model by encouraging the solutions to have smaller values in the weight vector $\mathbf{w}$.
- We do this by adding a weight penalty term to the error function:

$$E_{\mathrm{PLS}}(\mathbf{w}) := E_{\mathrm{LS}}(\mathbf{w}) + \lambda \, E_{\mathrm{W}}(\mathbf{w})$$

- A standard choice (to encourage small values) is the squared-weight penalty:

$$E_{\mathrm{W}}(\mathbf{w}) := \frac{1}{2} \sum_{m=1}^{M} w_m^2 = \frac{1}{2} \big\|\mathbf{w} \big\|^{2}_{2}$$

> **Aside:** This is a common form of regularisation; in the neural network literature it is sometimes referred to as "weight decay" or "L2 regularisation".

- The optimal solution to this problem now changes to:

$$\hat{\mathbf{w}}_{\mathrm{PLS}} = \big( \boldsymbol{\Phi}^{\top} \boldsymbol{\Phi} + \lambda \, \mathbf{I} \big)^{-1} \boldsymbol{\Phi}^{\top} \mathbf{y} $$

> **Aside:** In the statistics literature, this approach is often referred to as "ridge regression".

- Returning to the **demo** above, we can see that the situation is improved (we have reduced our *over-fitting*)

- Closer inspection, however, shows that the situation is more complicated. For some values of $\lambda$ we observe *"under-fitting"* where the model ignores the data in favour of an overly smooth result.

- We have a trade-off between the terms $E_{\mathrm{LS}}(\mathbf{w})$ and $E_{\mathrm{W}}(\mathbf{w})$ in terms of how well the function fits the data and how well it obeys our desire for the function to be smooth.

- Naturally, the question arises: *How to we select the best penalty parameter $\lambda$?*

## Validation Error

- A standard approach, for estimating this new hyperparameter $\lambda$, is to consider the performance on additional, unseen data as a proxy for the performance on test data that we will see in the future.

- The original observed data is divided into a **training dataset** and a **validation dataset**.

- The **demo** above provides the results for a sweep across the penalty parameter value (in log-space) against the mean squared error across three different datasets, the training, validation and test data. 

> **Note:** We keep other hyperparameter values fixed during this run; in particular we fix the noise level $\sigma_{\mathrm{noise}}$.

- The first thing we note is that the training error always decreases with the penalty parameter (hopefully this makes sense!) so we cannot use this as a measure to set the hyperparameter.

- The *validation error* seems a reasonable proxy for the test error but this has come at the expense of extra data. A new question arises; *How do we know how to split our data into the training and validation sets and is this a critical decision?*

# Bayesian Approach (inc Model Selection)

- The Bayesian approach is to first construct a probabilistic generative model of the observed data. The functional model we have been operating with is
$$y_n = f(x_n) + \epsilon_n$$
with our linear model $y(x ; \mathbf{w})$ and noise $\epsilon_n$.

- Given knowledge of the parameter $\mathbf{w}$, the prediction of the parameteric function is deterministic but the realisation of the noise is stochastic (random) since we have $\epsilon \sim \mathcal{N}(0, \sigma_{\mathrm{noise}}^{2})$.

- If we make an assumption that all the observations are drawn independently from some dataset, then the *likelihood* of the observations are given as:

\begin{align}
p(\mathbf{y} \mid \mathbf{w}, \mathbf{x}, \sigma_{\mathrm{noise}}^{2}) &= \prod_{n=1}^{N} p(y_n \mid \mathbf{w}, x_n, \sigma_{\mathrm{noise}}^{2}) \\
&= \frac{1}{(2 \pi \sigma_{\mathrm{noise}}^{2})^{-N/2}} \exp \left(- \frac{\|\mathbf{y} - \boldsymbol{\Phi}(\mathbf{x}) \, \mathbf{w}\|^{2}}{2 \sigma_{\mathrm{noise}}^{2}} \right) 
\end{align}

> **Aside:** The iid assumption is also something we take as given but is worthy of further discussion and consideration, in particular involving bias and representation in datasets.

- **Importantly:** The maximum liklihood solution (i.e. the value of $\mathbf{w}$ that maximises $p(\mathbf{y} \mid \mathbf{w}, \sigma_{\mathrm{noise}}^{2})$ is the same as the least-squares solution $\hat{\mathbf{w}}_{\mathrm{LS}}$. We can see this by taking the log of the likelihood (the log function is monotonic and therefore doesn't change the maximum) and noting that the expression inside the exponential is the least squares form. 

- In a similar manner, we can introduce a prior over the weight parameter (which will express our desire for smooth functions) as a zero-mean Gaussian distribution with a precision of $\alpha$:

$$p(\mathbf{w} \mid \alpha) = \prod_{m=1}^{M} \mathcal{N}\left(w_m \,\bigg\vert\, 0, \frac{1}{\alpha}\right) = \prod_{m=1}^{M} \left( \frac{\alpha}{2 \pi}\right)^{1/2} \exp \left(- \frac{\alpha w_m^2}{2} \right)$$

> **Aside:** This is a key part of the Bayesian approach, we express any knowledge we have before we see the data as a prior in our model. The rules of probablility then tell us how to integrate this knowledge to update our beliefs when combined with new observations or data.

- These two components (the likelihood and the prior) can now be combined using Bayes' Rule to give us our belief about the parameter vector given data (the **posterior** distribution over $\mathbf{w}$):

$$p(\mathbf{w} \mid \mathbf{y}, \mathbf{x}, \alpha, \sigma_{\mathrm{noise}}^{2}) = \frac{\text{likelihood} \times \text{prior}}{\text{evidence}} = \frac{p(\mathbf{y} \mid \mathbf{w}, \mathbf{x}, \sigma_{\mathrm{noise}}^{2}) \, p(\mathbf{w} \mid \alpha)}{p(\mathbf{y} \mid \mathbf{x}, \alpha, \sigma_{\mathrm{noise}}^{2})}$$

- As the prior is *conjugate*, the posterior has a closed form solution (the marginal product of two Gaussian distributions yields another Gaussian distribution) and so we can find the solution as:

$$
p(\mathbf{w} \mid \mathbf{y}, \mathbf{x}, \alpha, \sigma_{\mathrm{noise}}^{2}) =
\mathcal{N}(\boldsymbol{\mu}, \boldsymbol{\Sigma})
$$

\begin{align}
\text{with } \boldsymbol{\mu} &= \big( \boldsymbol{\Phi}^{\top} \boldsymbol{\Phi} + \alpha \sigma_{\mathrm{noise}}^{2} \, \mathbf{I} \big)^{-1} \boldsymbol{\Phi}^{\top} \mathbf{y} \\
\text{and } \boldsymbol{\Sigma} &= \sigma_{\mathrm{noise}}^{2} \big( \boldsymbol{\Phi}^{\top} \boldsymbol{\Phi} + \alpha \sigma_{\mathrm{noise}}^{2} \, \mathbf{I} \big)^{-1}
\end{align}

> **Note:** The posterior mean $\boldsymbol{\mu}$ is the same as the penalised least squares result $\hat{\mathbf{w}}_{\mathrm{PLS}}$ with $\lambda \equiv \alpha \sigma_{\mathrm{noise}}^{2}$. This is not a coincidence, but the fact that the mean of a Gaussian distribution is also the mode (most likely value) and the PLS estimate is equivalent to the point estimate (single value) that maximises the posterior - termed the MAP estimate (for Maximum a Posteriori). In contrast to the maximum likelihood estimate, this takes into account the prior as well - the Gaussian prior on $\mathbf{w}$ is equivalent to the squared penalty in the PLS estimate.

- To obtain a prediction for a new input, we need to consider a **weighted average of all the possible values of the parameter $\mathbf{w}$ where the weight is given by the plausibility that the data is explained by that value, i.e. the posterior over $\mathbf{w}$.**

- Therefore, the *predictive distribution* for a new input $x^{*}$ is:

\begin{align}
p(y^{*}\mid x^{*}, \mathbf{x}, \mathbf{y}, \alpha, \sigma_{\mathrm{noise}}^2) &= \int_{\mathbf{w}} p(x^{*}\mid\mathbf{w}, \mathbf{x}) \; p(\mathbf{w}\mid\mathbf{y},\alpha,\sigma_{\mathrm{noise}}^2) \; d \mathbf{w} \\
&= \mathcal{N}\big(x^{*} \,\big\vert\, \boldsymbol{\Phi}^{*} A^{-1}\boldsymbol{\Phi}^\top\mathbf{y}, \sigma_{\mathrm{noise}}^2 \boldsymbol{\Phi}^{*} A^{-1} \boldsymbol{\Phi}^{*\top} + \sigma_{\mathrm{noise}}^2\mathbf{I}\big)\\
\text{with } A &= [\boldsymbol{\Phi}^\top\boldsymbol{\Phi} + \sigma_{\mathrm{noise}}^2 \alpha \, \mathbf{I}]
\end{align}

- Going back to the **demo**, we can see this in action where the Bayesian version produces a distribution over plausible values (rather than a single estimate). We can see that this estimate is *well calibrated* for appropriate $\lambda$ (or conversely $\alpha = \lambda / \sigma_{\mathrm{noise}}^2)$ in the sense that the error bars cover the observed data (and the true function) appropriately.

- In addition, we see that we can use the evidence of the model (the denominator from Bayes' Rule), $p(\mathbf{y} \mid \mathbf{x}, \alpha, \sigma_{\mathrm{noise}}^{2})$ to determine the appropriate choice of the hyperparameters (in this case $\alpha$). 

\begin{align}
p(\mathbf{y} \mid \mathbf{x}, \alpha, \sigma_{\mathrm{noise}}^{2}) 
&= \int_{\mathbb{R}^M}  p(\mathbf{y} \mid \mathbf{w}, \mathbf{x}, \sigma_{\mathrm{noise}}^{2}) \, p(\mathbf{w} \mid \alpha) \,   d\mathbf{w} \\
&= \mathcal{N}\left( \mathbf{y} \,\middle\vert\, \mathbf{0}, \left[ \frac{\boldsymbol{\Phi} \boldsymbol{\Phi}^\top}{\alpha} + \sigma_{\mathrm{noise}}^{2} \mathbf{I} \right] \right)
\end{align}

- This evidence term is obtained by marginalising out the parameters $\mathbf{w}$ so we are left with a value that we can use to determine $\alpha$ irrespectively of the values of $\mathbf{w}$. This is a critical part of the Bayesian workflow and is an example of **Model Selection**; we can consider different choices of the prior on $\alpha$ as different model proposals and we wish to evaluate between them. 

- In contrast to the validation approach with penalised least squares, we get obtain an estimate without having to use additional data or decide how to split the data into training and validation sets; we simply treat all the observed data equally.

<!--
\begin{frame}{Bayesian Regression Revisited}
\begin{itemize}
\item If we assume an iid Gaussian noise model for $\varepsilon_n \sim \mathcal{N}(0, \sigma^2)$ then we have the likelihood 
$$p(\mathbf{t}\mid\mathbf{w}, \sigma^2) = \prod_{n=1}^{N} p(t_n \mid \mathbf{w}, \sigma^2)$$
with
$$p(t_n \mid \mathbf{w}, \sigma^2) = \mathcal{N}(f(x_n; \mathbf{w}), \sigma^2)$$
\item  We put a zero mean Gaussian prior on the weights
$$p(\mathbf{w}\mid\alpha) = \prod_{m=1}^{M} \mathcal{N}(w_m \mid 0, \alpha^{-1})$$
\item with precision (inverse variance) $\alpha$
\end{itemize}
\end{frame}

\begin{frame}{Bayesian Regression Revisited}
\begin{itemize}
\item Since Gaussians are conjugate to themselves, the posterior over the weights is also Gaussian and may be found in closed form as
$$p(\mathbf{w}\mid\mathbf{t},\alpha,\sigma^2) = \mathcal{N}(\mathbf{w}\mid\boldsymbol{\mu}, \boldsymbol{\Sigma})$$
\begin{align*}
\boldsymbol{\mu} &= [\boldsymbol{\Phi}^\Trans\boldsymbol{\Phi} + \sigma^2 \alpha \mathbf{I}]^{-1}\boldsymbol{\Phi}^\Trans\mathbf{t}\\
\boldsymbol{\Sigma} &= \sigma^2 [\boldsymbol{\Phi}^\Trans\boldsymbol{\Phi} + \sigma^2 \alpha \mathbf{I}]^{-1}
\end{align*}
\item Therefore, the \keyterm{predictive distribution} for a new input $x^{*}$ is
%
\begin{align*}
p(x^{*}\mid\mathbf{x}, \mathbf{t}, \alpha, \sigma^2) &= \int_{\mathbf{w}} p(x^{*}\mid\mathbf{w}, \mathbf{x}) \; p(\mathbf{w}\mid\mathbf{t},\alpha,\sigma^2) \; d \mathbf{w} \\
&= \mathcal{N}(x^{*} \mid \boldsymbol{\Phi}^{*} A^{-1}\boldsymbol{\Phi}^\Trans\mathbf{t}, \boldsymbol{\Phi}^{*} A^{-1} \boldsymbol{\Phi}^{*\Trans} + \sigma^2\mathbf{I})\\
\text{with} \; A &= [\boldsymbol{\Phi}^\Trans\boldsymbol{\Phi} + \sigma^2 \alpha \mathbf{I}]\\
\text{and} \; \boldsymbol{\Phi}^{*}_{m}  &= \phi_{m} (x^{*})
\end{align*}
%
\end{itemize}
\end{frame}-->